In [5]:
import pandas as pd
from dateutil import parser
import os
from cassandra.cluster import Cluster
import logging 
from time import time

download_dir = "X:\\Data_Analytics\\FNO\\Download\\"
master_dir = "X:\\Data_Analytics\\FNO\\Master\\"
log_path = "X:\\Data_Analytics\\FNO\\"
output_dir = 'X:\\Data_Analytics\\FNO\\Output\\'

# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")


# create python cluster object to connect to your cassandra cluster (specify ip address of nodes to connect within your cluster)
cluster = Cluster(['127.0.0.1'])
logging.info('Cassandra Cluster connected...')
# connect to your keyspace and create a session using which u can execute cql commands 
session = cluster.connect('test_df')
logging.info('Using test_df keyspace')

# CREATE A TABLE; dump bhavcopies to this table
session.execute('CREATE TABLE IF NOT EXISTS test_bhavcopy (SYMBOL VARCHAR,INSTRUMENT VARCHAR, EXPIRY_DT DATE, STRIKE_PR DECIMAL, OPTION_TYP VARCHAR, OPEN DECIMAL, HIGH DECIMAL,LOW DECIMAL, CLOSE DECIMAL, SETTLE_PR DECIMAL, CONTRACTS DECIMAL, VAL_INLAKH DECIMAL, OPEN_INT DECIMAL, CH_IN_OI DECIMAL, PRICE_DATE DATE, KEY VARCHAR, PRIMARY KEY (KEY, PRICE_DATE))')


def expiry_encodings(bhav_copy):  
    '''Function for expiry encodings'''
    
    expiry_contract_dates = pd.read_excel(master_dir+'Expiry_contract_Master.xlsx')
    expiry_contract_dates.set_index('Date',inplace=True)
    expiry_contract_dates.index.name = 'TIMESTAMP'
    FINAL = bhav_copy.merge(expiry_contract_dates, on='TIMESTAMP')
    FINAL['expiry_number'] = '0'
    for c in '321':
        FINAL.loc[FINAL['EXPIRY_DT'] == FINAL['Expiry'+c], 'expiry_number'] = c
        
    FINAL['Expiry_encodings'] = FINAL['SYMBOL'].astype(str) + '_' + \
    FINAL['INSTRUMENT'].astype(str) + '_' + FINAL['STRIKE_PR'].astype(str) + \
    '_' + FINAL['OPTION_TYP'].astype(str) +' _' + FINAL['expiry_number']
        
    
    
    # append zeros for the expiry dates that were not found in master 
    #FINAL.loc[FINAL.Expiry_encodings.str.endswith('0'), 'Expiry_encodings'] = 0
    
    FINAL.drop(columns=['Expiry1','Expiry2','Expiry3','expiry_number'], inplace= True)
    
    return FINAL


def main():
    
    # walk through all the downlaoded bhavcopies in downlaods dir
    for r,d,f in os.walk(download_dir):
        # read master files for symbols 
        logging.info('Reading master file for symbols')
        master_df = pd.read_excel(master_dir+'MasterData.xlsx')
        master_df = master_df['SYMBOL']

        # read expiry contracts master for expiry encodings    
        logging.info('Reading expiry_contract_master file for expiry encodings')
        expiry_encodings_df = pd.read_excel(master_dir+'Expiry_contract_Master.xlsx')      

        # process every bhavcopy in expiry encodings format
        logging.info('Traverse through each bhavcopy')
        for csv_file in f:
            #read each csv file 
            print csv_file
            file_name = csv_file
            logging.info('Processing bhavcopy {0}...'.format(file_name))

            bhav_copy = pd.read_csv(download_dir + file_name)
            bhav_copy.dropna(axis=1, inplace=True)

            # convert to cassandra date format
            bhav_copy['EXPIRY_DT'] = pd.to_datetime(bhav_copy['EXPIRY_DT'], format='%d-%b-%Y')
            bhav_copy['TIMESTAMP'] = pd.to_datetime(bhav_copy['TIMESTAMP'], format='%d-%b-%Y')
            bhav_copy.set_index('SYMBOL', inplace= True)

            # Consider SYMBOL present in master file, exclude rest    
            master_df = pd.read_excel(master_dir+'MasterData.xlsx')
            master_df = master_df['SYMBOL']
            logging.info('No match for {0} symbols in master file'.format(bhav_copy.loc[~bhav_copy.index.isin(master_df)].index))
            bhav_copy = bhav_copy.loc[bhav_copy.index.isin(master_df)]
            bhav_copy.reset_index(inplace=True)
            
            # Get expiry encodings
            bhav_copy = expiry_encodings(bhav_copy)
            bhav_copy.index 
            
            # write processed bhav_copy to temp file
            bhav_copy.to_csv('temp.csv', index= False)
            
            # write csv file to cassandra db
            os.system("X:\\Data_Analytics\\FNO\\dump.bat ")
            
                    


           
        
        
        
        
start_time = time()                    
if __name__ == '__main__':
    main()
end_time = time()

print "Execution time: {0} Seconds.... ".format(end_time - start_time)
    
        
        
        

fo22OCT2018bhav.csv
Execution time: 16.5740001202 Seconds.... 


In [21]:
bhav_copy = pd.read_csv(download_dir+file_nam)
bhav_copy.dropna(axis=1, inplace=True)
        
# convert to cassandra date format
bhav_copy['EXPIRY_DT'] = pd.to_datetime(bhav_copy['EXPIRY_DT'], format='%d-%b-%Y')
bhav_copy['TIMESTAMP'] = pd.to_datetime(bhav_copy['TIMESTAMP'], format='%d-%b-%Y')
bhav_copy.set_index('SYMBOL', inplace= True)

master_df = pd.read_excel(master_dir+'MasterData.xlsx')
master_df = master_df['SYMBOL']

print bhav_copy.loc[~bhav_copy.index.isin(master_df)].index
bhav_copy = bhav_copy.loc[bhav_copy.index.isin(master_df)]

bhav_copy.reset_index(inplace=True)

Index([u'INDIAVIX', u'INDIAVIX'], dtype='object', name=u'SYMBOL')


In [154]:

expiry_contract_dates = pd.read_excel(master_dir+'Expiry_contract_Master.xlsx')
expiry_contract_dates.set_index('Date',inplace=True)
expiry_contract_dates.index.name = 'TIMESTAMP'

FINAL = bhav_copy.merge(expiry_contract_dates, on='TIMESTAMP')

FINAL['Expiry_encodings'] = FINAL.apply(lambda row: '{0}_{1}_{2}_{3}_{4}'.format(row['SYMBOL'],row['INSTRUMENT'],row['STRIKE_PR'],row['OPTION_TYP'], encoder(row['EXPIRY_DT'],
                                                                                                                                             row['Expiry1'],
                                                                                                                                             row['Expiry2'],
                                                                                                                                             row['Expiry3'])), axis =1)

In [240]:
def descriptive_func(bhav_copy):
    '''Function to get Min Max Avg 95% 97% 5% 3% for individual stocks'''
    #bhav_copy = pd.read_csv('fo22OCT2018bhavtemp.csv')
    bhav_copy = bhav_copy
    
    instrument = ['FUTSTK','FUTIDX']
    #result = pd.DataFrame(columns=['SYMBOL','INSTRUMENT','Min','Max','Avg','95%','97%','5%','3%'])
    result = []
    
    logging.info('Generating report for CHG_IN_OI...Min/Max/Avg/95%/97%/5%/3%')
    for instru in instrument:
        
        process = bhav_copy[bhav_copy['INSTRUMENT']==instru][['SYMBOL','INSTRUMENT','OPEN_INT','Expiry_encodings']]
        symbol = list(set(process['SYMBOL']))
        
        for symbo in symbol:
            mi,ma,avg,per95,per97,per5,per3 = process[(process['SYMBOL']==symbo) & 
                                                        (process['INSTRUMENT']==instru) &
                                                        (process.Expiry_encodings.str.endswith('1'))]['OPEN_INT'].describe(
                percentiles=[.95,.97,.05,.03]).loc[['min','max','mean','95%','97%','5%','3%']].values            
            
            
            result.append([symbo,instru,mi,ma,avg,per95,per97,per5,per3])                    
                          
                          
    result = pd.DataFrame(result, columns=['SYMBOL','INSTRUMENT','Min','Max','Avg','95%','97%','5%','3%']) 
    result.to_excel(output_dir+'Descriptive_report.xlsx',index=False)

    logging.info('Report successfully generated...')
    return result
  


from time import time
start = time()

result = descriptive_func(bhav_copy)

end = time()
print end-start

1.75999999046


In [67]:
import pandas as pd
from dateutil import parser
import os
from cassandra.cluster import Cluster
import logging 
from time import time

download_dir = "X:\\Data_Analytics\\FNO\\Download\\"
master_dir = "X:\\Data_Analytics\\FNO\\Master\\"
log_path = "X:\\Data_Analytics\\FNO\\"

# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")


# create python cluster object to connect to your cassandra cluster (specify ip address of nodes to connect within your cluster)
cluster = Cluster(['127.0.0.1'])
logging.info('Cassandra Cluster connected...')
# connect to your keyspace and create a session using which u can execute cql commands 
session = cluster.connect('test_df')
logging.info('Using test_df keyspace')

# CREATE A TABLE; dump bhavcopies to this table
session.execute('CREATE TABLE IF NOT EXISTS test_bhavcopy (SYMBOL VARCHAR,INSTRUMENT VARCHAR, EXPIRY_DT DATE, STRIKE_PR DECIMAL, OPTION_TYP VARCHAR, OPEN DECIMAL, HIGH DECIMAL,LOW DECIMAL, CLOSE DECIMAL, SETTLE_PR DECIMAL, CONTRACTS DECIMAL, VAL_INLAKH DECIMAL, OPEN_INT DECIMAL, CH_IN_OI DECIMAL, PRICE_DATE DATE, KEY VARCHAR, PRIMARY KEY (KEY, PRICE_DATE))')



def encoder(expiry_dt,expiry1,expiry2,expiry3,symbol,instrument,strike_pr,option_typ):
    if expiry_dt == expiry1:
        return 1
    elif expiry_dt == expiry2:
        return 2
    elif expiry_dt == expiry3:
        return 3
    else:
        logging.info('Expiry date {0} for /{1} / {2} / {3} / {4}/ was not found.'.format(expiry_dt,symbol,instrument,strike_pr,option_typ))
        return 0
    
      
    

def expiry_encodings(bhav_copy):  
    '''Function for expiry encodings'''
    
    expiry_contract_dates = pd.read_excel(master_dir+'Expiry_contract_Master.xlsx')
    expiry_contract_dates.set_index('Date',inplace=True)
    expiry_contract_dates.index.name = 'TIMESTAMP'
    FINAL = bhav_copy.merge(expiry_contract_dates, on='TIMESTAMP')
    FINAL['Expiry_encodings'] = FINAL.apply(lambda row: '{0}_{1}_{2}_{3}_{4}'.format(row['SYMBOL'],row['INSTRUMENT'],
                                                                                     row['STRIKE_PR'],row['OPTION_TYP'],
                                                                                     encoder(row['EXPIRY_DT'],
                                                                                             row['Expiry1'],
                                                                                             row['Expiry2'],
                                                                                             row['Expiry3'],
                                                                                            row['SYMBOL'],row['INSTRUMENT'],
                                                                                            row['STRIKE_PR'],row['OPTION_TYP'])), axis =1)
    # Exclude expiry encodings with zero 
    FINAL = FINAL[(FINAL.Expiry_encodings.str.endswith('1')) | 
                  (FINAL.Expiry_encodings.str.endswith('2')) |
                  (FINAL.Expiry_encodings.str.endswith('3'))]  
    
    FINAL.drop(columns=['Expiry1','Expiry2','Expiry3'], inplace= True)
    
   
    return FINAL


def main():
    
    # walk through all the downlaoded bhavcopies in downlaods dir
    for r,d,f in os.walk(download_dir):
        # read master files for symbols 
        logging.info('Reading master file for symbols')
        master_df = pd.read_excel(master_dir+'MasterData.xlsx')
        master_df = master_df['SYMBOL']

        # read expiry contracts master for expiry encodings    
        logging.info('Reading expiry_contract_master file for expiry encodings')
        expiry_encodings_df = pd.read_excel(master_dir+'Expiry_contract_Master.xlsx')      

        # process every bhavcopy in expiry encodings format
        logging.info('Traverse through each bhavcopy')
        for csv_file in f:
            #read each csv file 
            print csv_file
            file_name = csv_file
            logging.info('Processing bhavcopy {0}...'.format(file_name))

            bhav_copy = pd.read_csv(download_dir + file_name)
            bhav_copy.dropna(axis=1, inplace=True)

            # convert to cassandra date format
            bhav_copy['EXPIRY_DT'] = pd.to_datetime(bhav_copy['EXPIRY_DT'], format='%d-%b-%Y')
            bhav_copy['TIMESTAMP'] = pd.to_datetime(bhav_copy['TIMESTAMP'], format='%d-%b-%Y')
            bhav_copy.set_index('SYMBOL', inplace= True)

            # Consider SYMBOL present in master file, exclude rest    
            master_df = pd.read_excel(master_dir+'MasterData.xlsx')
            master_df = master_df['SYMBOL']
            logging.info('No match for {0} symbols in master file'.format(bhav_copy.loc[~bhav_copy.index.isin(master_df)].index))
            bhav_copy = bhav_copy.loc[bhav_copy.index.isin(master_df)]
            bhav_copy.reset_index(inplace=True)
            
            # Get expiry encodings
            bhav_copy = expiry_encodings(bhav_copy)
            bhav_copy.index 
            
            # write processed bhav_copy to temp file
            bhav_copy.to_csv('temp.csv', index= False)
            
            # write csv file to cassandra db
            os.system("X:\\Data_Analytics\\FNO\\dump.bat ")   
            # remove bhavcopy from download dir
            logging.info('Removed bhavcopy {0} from download dir'.format(file_name))
            os.remove(download_dir + file_name)
            #os.remove('temp.csv')
            

                   
        
        
        
start_time = time()                    
if __name__ == '__main__':
    main()
end_time = time()

print "Execution time: {0} Seconds.... ".format(end_time - start_time)

        

fo15NOV2018bhav.csv
fo16NOV2018bhav.csv
fo19NOV2018bhav.csv
fo20NOV2018bhav.csv
fo21NOV2018bhav.csv
fo22NOV2018bhav.csv
Execution time: 140.451999903 Seconds.... 


In [ ]:
def descriptive_func(bhav_copy):
    '''Function to get Min Max Avg 95% 97% 5% 3% for individual stocks'''
    #bhav_copy = pd.read_csv('fo22OCT2018bhavtemp.csv')
    bhav_copy = bhav_copy
    
    instrument = ['FUTSTK','FUTIDX']
    #result = pd.DataFrame(columns=['SYMBOL','INSTRUMENT','Min','Max','Avg','95%','97%','5%','3%'])
    result = []
    
    logging.info('Generating report for CHG_IN_OI...Min/Max/Avg/95%/97%/5%/3%')
    for instru in instrument:
        
        process = bhav_copy[bhav_copy['INSTRUMENT']==instru][['SYMBOL','INSTRUMENT','OPEN_INT','Expiry_encodings']]
        symbol = list(set(process['SYMBOL']))
        
        for symbo in symbol:
            mi,ma,avg,per95,per97,per5,per3 = process[(process['SYMBOL']==symbo) & 
                                                        (process['INSTRUMENT']==instru) &
                                                        (process.Expiry_encodings.str.endswith('1'))]['OPEN_INT'].describe(
                percentiles=[.95,.97,.05,.03]).loc[['min','max','mean','95%','97%','5%','3%']].values            
            
            
            result.append([symbo,instru,mi,ma,avg,per95,per97,per5,per3])                    
                          
                          
    result = pd.DataFrame(result, columns=['SYMBOL','INSTRUMENT','Min','Max','Avg','95%','97%','5%','3%']) 
    result.to_excel(output_dir+'Descriptive_report.xlsx',index=False)

    logging.info('Report successfully generated...')
    return result
  


from time import time
start = time()

result = descriptive_func(bhav_copy)

end = time()
print end-start

In [147]:
import pandas as pd
from dateutil import parser
import os
from cassandra.cluster import Cluster
import logging 
from time import time
import datetime
import cassandra

download_dir = "X:\\Data_Analytics\\FNO\\Download\\"
master_dir = "X:\\Data_Analytics\\FNO\\Master\\"
log_path = "X:\\Data_Analytics\\FNO\\"
output_dir = "X:\Data_Analytics\FNO\Output\\"

# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")


# Define a pandas factory to read the cassandra data into pandas dataframe:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)



# create python cluster object to connect to your cassandra cluster (specify ip address of nodes to connect within your cluster)
cluster = Cluster(['127.0.0.1'])
logging.info('Cassandra Cluster connected...')
# connect to your keyspace and create a session using which u can execute cql commands 
session = cluster.connect('test_df')
logging.info('Using test_df keyspace')
#define rows fetched from cassandra to be a pandas dataframe
session.row_factory = pandas_factory
session.default_fetch_size = None

# Process for date range 
#from_date = parser.parse(raw_input('Enter from price date :'))
#to_date = parser.parse(raw_input('Enter to price date :'))
#logging.info('Processing report from price date {0} to {1} price date...'.format(from_date,to_date))


logging.info('Read master file to fetch Active FNO symbols and generate keys...')

keys = pd.read_excel(master_dir+'MasterData.xlsx')
keys = keys[keys['IsActiveFNO']==True]
keys.loc[keys['Type']=='SSF','instrument'] = 'FUTSTK'
keys.loc[keys['Type']=='Index','instrument'] = 'FUTIDX'
keys = keys['SYMBOL']+"_"+keys['instrument']+"_0.0_XX_1"

# create empty dataframe 
bhav_copy = pd.DataFrame()

for key in keys:
    
    query = 'SELECT symbol,instrument,open_int,key,price_date,expiry_dt FROM test_bhavcopy WHERE price_date >= \'{0}\' AND price_date <= \'{1}\' AND key = \'{2}\'ALLOW FILTERING;'.format(
            datetime.datetime.now().date() - datetime.timedelta(days=365),datetime.datetime.now().date(),key)
    rows = session.execute(query, timeout = None)
    rows = rows._current_rows
    
    if len(rows)==0:
        pass
    else :
        
        bhav_copy = bhav_copy.append(rows,ignore_index=True)
                
logging.info('Calling descriptive func to process descriptive stats') 
descriptive_func(bhav_copy,keys)


          


In [2]:
def instu(row):
    row = row.split('_')
    return row[1]

def sym(row):
    row = row.split('_')
    return row[0]

def descriptive_func(bhav_copy,keys):
    '''Function to get Min Max Avg 95% 97% 5% 3% for individual stocks'''      
 
    logging.info('Generating report for OPEN_INT...Min/Max/Avg/95%/97%/5%/3%')
    result = []
   
    
    for key in keys:
        
        mi,ma,avg,per95,per97,per5,per3 = bhav_copy[bhav_copy['key']==key]['open_int'].astype(int).describe(
            percentiles=[.95,.97,.05,.03]).loc[['min','max','mean','95%','97%','5%','3%']].values            
            
        #print [ bhav_copy[bhav_copy['key']==key]['symbol'][0],bhav_copy[bhav_copy['key']==key]['instrument'][0],mi,ma,avg,per95,per97,per5,per3]
            
        result.append([key,mi,ma,avg,per95,per97,per5,per3])     
       
                          
                          
                      
    result = pd.DataFrame(result, columns=['key','Min','Max','Avg','95%','97%','5%','3%']) 
    # get symbol from key
    result['SYMBOL'] = result.apply(lambda row: sym(row['key']), axis=1)
    # get instruments
    result['INSTRUMENT'] = result.apply(lambda row: instu(row['key']), axis=1)
    # drop key
    result.drop('key',inplace=True, axis=1)
    # write the report to excel 
    result = result[['SYMBOL','INSTRUMENT','Min','Max','Avg','95%','97%','5%','3%']]
    result.to_excel(output_dir+'Descriptive_report.xlsx', index=False)
    logging.info('Descriptive report generated successfully in excel...')
    


In [53]:
query = 'SELECT * FROM test_bhavcopy WHERE price_date = \'{0}\' AND symbol = \'{1}\' and (instrument =\'{2}\' OR instrument =\'{3}\') ALLOW FILTERING;'.format(
    datetime.datetime.date(from_date),symbo,instrument[0],instrument[1])

rows = session.execute(query, timeout = None)


SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:105 mismatched input 'OR' expecting ')' (...'ACC' and (instrument ='FUTSTK' [OR]...)">

In [138]:
from_date = parser.parse(raw_input('Enter from price date :'))
to_date = parser.parse(raw_input('Enter to price date :'))

Enter from price date :1 oct 2017
Enter to price date :1 oct 2018


In [10]:
query

"SELECT * FROM test_bhavcopy WHERE price_date >= '2017-10-01' AND price_date <= '2018-10-01' ALLOW FILTERING;"

In [121]:
rows._current_rows

key  price_date ch_in_oi   close contracts   expiry_dt  \
0  ACC_FUTSTK_0.0_XX_2  2018-10-01     2400  1576.9        36  2018-11-29   
1  ACC_FUTSTK_0.0_XX_3  2018-10-01        0  1591.1         0  2018-12-27   
2  ACC_FUTSTK_0.0_XX_1  2018-10-01    43600  1572.8      4630  2018-10-25   

      high instrument      low    open open_int option_typ settle_pr  \
0  1582.95     FUTSTK   1533.7  1561.4    11200         XX    1576.9   
1      0.0     FUTSTK      0.0     0.0        0         XX   1602.15   
2   1585.0     FUTSTK  1520.45  1550.6  1338000         XX    1572.8   

  strike_pr symbol val_inlakh  
0       0.0    ACC     224.54  
1       0.0    ACC        0.0  
2       0.0    ACC   28839.72

In [141]:
from_date = parser.parse(raw_input('Enter from price date :'))
to_date = parser.parse(raw_input('Enter to price date :'))
logging.info('Using test_df keyspace')
query = 'SELECT symbol,instrument,open_int,key FROM test_bhavcopy WHERE price_date >= \'{0}\' AND price_date <= \'{1}\' AND key = \'{2}\'ALLOW FILTERING;'.format(
            datetime.datetime.date(from_date),datetime.datetime.date(to_date),'RELIANCE_FUTSTK_0.0_XX_1')
rows = session.execute(query,timeout = None)
bhav = rows._current_rows
bhav = bhav[bhav.key.str.endswith('1')]


Enter from price date :1 oct 2017
Enter to price date :1 oct 2018


Enter from price date :26 nov 2017
Enter to price date :26 nov 2018


In [56]:
for key in keys:
    print bhav_copy[bhav_copy['key']==key]['symbol']

ACC


KeyError: 0L

In [60]:
symbols,instrument = symbols['SYMBOL'],symbols['']

Exchange   BloomCode      SYMBOL  IsNifty         Sector   Type  \
0        NSE         ACC         ACC    False         Cement    SSF   
1        NSE         ADE    ADANIENT    False    Diversified    SSF   
2        NSE       ADSEZ  ADANIPORTS     True          Infra    SSF   
3        NSE       ADANI  ADANIPOWER    False          Power    SSF   
4        NSE         AJP  AJANTPHARM    False         Pharma    SSF   
5        NSE        ALBK        ALBK    False       PSUBANKS    SSF   
6        NSE        AMRJ  AMARAJABAT    False      chemicals    SSF   
7        NSE        ACEM   AMBUJACEM    False         Cement    SSF   
8        NSE        APHS  APOLLOHOSP    False         Pharma    SSF   
9        NSE        APTY  APOLLOTYRE    False           Auto    SSF   
10       NSE       ARVND      ARVIND    False       Consumer    SSF   
11       NSE          AL    ASHOKLEY    False           Auto    SSF   
12       NSE        APNT  ASIANPAINT     True       Consumer    SSF   
13       NSE        ARBP  AUROPHARMA    False         Pharma    SSF   
14       NSE        AXSB    AXISBANK     True  Private Banks    SSF   
15       NSE       BJAUT  BAJAJ-AUTO     True           Auto    SSF   
16       NSE         BAF  BAJFINANCE     True     Financials    SSF   
17       NSE         BOB  BANKBARODA    False       PSUBANKS    SSF   
18       NSE         BOI   BANKINDIA    False       PSUBANKS    SSF   
19       NSE        BATA   BATAINDIA    False       Consumer    SSF   
20       NSE         BHE         BEL    False  Capital Goods    SSF   
21       NSE        BEML        BEML    False  Capital Goods    SSF   
22       NSE       BHAFI   BHARATFIN    False     Financials    SSF   
23       NSE        BHFC  BHARATFORG    False           Auto    SSF   
24       NSE      BHARTI  BHARTIARTL     True        Telecom    SSF   
25       NSE        BHEL        BHEL    False  Capital Goods    SSF   
26       NSE        BIOS      BIOCON    False         Pharma    SSF   
27       NSE         BOS    BOSCHLTD    False           Auto    SSF   
28       NSE        BPCL        BPCL     True         Energy    SSF   
29       NSE        BRIT   BRITANNIA    False       Consumer    SSF   
..       ...         ...         ...      ...            ...    ...   
181      NSE        NBCC        NBCC    False          Infra    SSF   
182      NSE       MAHGL         MGL    False         Energy    SSF   
183      NSE        BRGR  BERGEPAINT    False       Consumer    SSF   
184      NSE        CANF  CANFINHOME    False     Financials    SSF   
185      NSE        CIFC    CHOLAFIN    False     Financials    SSF   
186      NSE        GSFC        GSFC    False      chemicals    SSF   
187      NSE         MCX         MCX    False     Financials    SSF   
188      NSE        NEST   NESTLEIND    False       Consumer    SSF   
189      NSE        VGRD      VGUARD    False  Capital Goods    SSF   
190      NSE          GP  GODFRYPHLP    False       Consumer    SSF   
191      NSE       BJFIN  BAJAJFINSV     True     Financials    SSF   
192      NSE        MRPL        MRPL    False         Energy    SSF   
193      NSE          RW     RAYMOND    False       Consumer    SSF   
194      NSE         HCC         HCC    False          Infra    SSF   
195      NSE        TRCL    RAMCOCEM    False         Cement    SSF   
196      NSE        NACL  NATIONALUM    False         Metals    SSF   
197      NSE         RBK     RBLBANK    False  Private Banks    SSF   
198      NSE         MRL  CHENNPETRO    False         Energy    SSF   
199      NSE        SREI    SREINFRA    False          Infra    SSF   
200      NSE        MGFL  MANAPPURAM    False     Financials    SSF   
201      NSE        IPRU  ICICIPRULI    False     Financials    SSF   
202      NSE       REPCO   REPCOHOME    False     Financials    SSF   
203      NSE         KJC  KAJARIACER    False           None    SSF   
204      NSE   BANKNIFTY   BANKNIFTY    False          Index  Index   
205      NSE       NIFTY    

In [5]:
import smtplib

from string import Template

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

MY_ADDRESS = 'my_address@example.comm'
PASSWORD = 'mypassword'

def get_contacts(filename):
    """
    Return two lists names, emails containing names and email addresses
    read from a file specified by filename.
    """
    
    names = []
    emails = []
    with open(filename, mode='r', encoding='utf-8') as contacts_file:
        for a_contact in contacts_file:
            names.append(a_contact.split()[0])
            emails.append(a_contact.split()[1])
    return names, emails

def read_template(filename):
    """
    Returns a Template object comprising the contents of the 
    file specified by filename.
    """
    
    with open(filename, 'r', encoding='utf-8') as template_file:
        template_file_content = template_file.read()
    return Template(template_file_content)

def main():
    names, emails = get_contacts('mycontacts.txt') # read contacts
    message_template = read_template('message.txt')

    # set up the SMTP server
    s = smtplib.SMTP(host='your_host_address_here', port=your_port_here)
    s.starttls()
    s.login(MY_ADDRESS, PASSWORD)

    # For each contact, send the email:
    for name, email in zip(names, emails):
        msg = MIMEMultipart()       # create a message

        # add in the actual person name to the message template
        message = message_template.substitute(PERSON_NAME=name.title())

        # Prints out the message body for our sake
        print(message)

        # setup the parameters of the message
        msg['From']=MY_ADDRESS
        msg['To']=email
        msg['Subject']="This is TEST"
        
        # add in the message body
        msg.attach(MIMEText(message, 'plain'))
        
        # send the message via the server set up earlier.
        s.send_message(msg)
        del msg
        
    # Terminate the SMTP session and close the connection
    s.quit()
    
if __name__ == '__main__':
    main()

ConnectionError: HTTPSConnectionPool(host='ratesapi.io', port=443): Max retries exceeded with url: /api/latest?symbols=INR&base=USD (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000000111D4AC8>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed',))

In [82]:
query = 'SELECT symbol,instrument,open_int,key FROM test_bhavcopy WHERE price_date >= \'{0}\' AND price_date <= \'{1}\' AND key = \'{2}\'ALLOW FILTERING;'.format(
                datetime.datetime.date(from_date),datetime.datetime.date(to_date),'NIFTYCPSE_FUTIDX_0.0_XX_1')
rows = session.execute(query, timeout = None)
rows = rows._current_rows

#NIFTYCPSE_FUTIDX_0.0_XX_1

acc = acc.append(rows, ignore_index=True)

acc.to_excel('ACC.xlsx')

In [40]:
pd.to_datetime(sorted(bhav_copy[bhav_copy['key']=='ACC_FUTSTK_0.0_XX_1']['price_date']

TypeError: <class 'cassandra.util.Date'> is not convertible to datetime

In [5]:
bhav_copy[(bhav_copy['key']=='ACC_FUTSTK_0.0_XX_1') & (bhav_copy['price_date']==cassandra.util.Date(datetime.datetime.now()-datetime.timedelta(days=1)) ) ]

symbol instrument open_int                  key  price_date   expiry_dt
226    ACC     FUTSTK  1225200  ACC_FUTSTK_0.0_XX_1  2018-11-26  2018-11-29

In [68]:
cassandra.util.Date(datetime.datetime.now())

Date(17861)

In [146]:
def instu(row):
    row = row.split('_')
    return row[1]

def sym(row):
    row = row.split('_')
    return row[0]

def descriptive_func(bhav_copy,keys):
    '''Function to get Min Max Avg 95% 97% 5% 3% for individual stocks'''      
 
    logging.info('Generating report for OPEN_INT...Min/Max/Avg/95%/97%/5%/3%')
    result = []
    # current date
    current_date = sorted(bhav_copy['price_date'], reverse = True)[0]
    logging.info('Processing for current date: {0}'.format(current_date))
    
    for key in keys:
        current_OI = 0
        
        # drop rows with zero OI
        stock = bhav_copy[(bhav_copy['key']==key) & (~(bhav_copy['open_int']==0))]
        #current OI
        if len(stock[stock['price_date']==current_date ]) == 0:  
            logging.info('{0} is newly traded/removed from FNO...'.format(key.split("_")[0]))
            current_OI = 0
        else:
            current_OI = int(stock[stock['price_date']==current_date]['open_int'].values[0])
        
                 
        try:
            # convert to int and get the percentiles with describe method
            count,mi,ma,avg,per95,per97,per5,per3 = stock['open_int'].astype(int).describe(
                percentiles=[.95,.97,.05,.03]).loc[['count','min','max','mean','95%','97%','5%','3%']].values 
            
        except Exception as e:
            logging.info('{0}'.format(e))
            logging.info('{0} Traded newly with OI zero...'.format(key.split("_")[0]))
        
        
        result.append([key,count,mi,ma,avg,per95,per97,per5,per3,current_OI])     
       
                          
                          
                      
    result = pd.DataFrame(result, columns=['key','Days_traded','Min','Max','Avg','95%','97%','5%','3%','Current_OI']) 
    # get symbol from key
    result['SYMBOL'] = result.apply(lambda row: sym(row['key']), axis=1)
    # get instruments
    result['INSTRUMENT'] = result.apply(lambda row: instu(row['key']), axis=1)
    # drop key
    result.drop('key',inplace=True, axis=1)
    # write the report to excel 
    result = result[['SYMBOL','INSTRUMENT','Days_traded','Min','Max','Avg','95%','97%','5%','3%','Current_OI']]
    result.to_excel(output_dir+'Descriptive_report.xlsx', index=False)
    logging.info('Descriptive report generated successfully in excel...')
   
    


In [27]:
np.percentile(int(bhav_copy[(bhav_copy['key']=='NIFTYCPSE_FUTIDX_0.0_XX_1') & (~(bhav_copy['open_int']==0))]['open_int'].values),0.95)

2750.0

In [35]:
bhav_copy[bhav_copy['key']=='NIFTYCPSE_FUTIDX_0.0_XX_1']['open_int'].astype(int).describe()

count     207.000000
mean       13.285024
std       191.138213
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      2750.000000
Name: open_int, dtype: float64

In [36]:
bhav_copy[bhav_copy['key']=='NIFTYCPSE_FUTIDX_0.0_XX_1']['open_int']

47630       0
47631       0
47632       0
47633    2750
47634       0
47635       0
47636       0
47637       0
47638       0
47639       0
47640       0
47641       0
47642       0
47643       0
47644       0
47645       0
47646       0
47647       0
47648       0
47649       0
47650       0
47651       0
47652       0
47653       0
47654       0
47655       0
47656       0
47657       0
47658       0
47659       0
         ... 
47807       0
47808       0
47809       0
47810       0
47811       0
47812       0
47813       0
47814       0
47815       0
47816       0
47817       0
47818       0
47819       0
47820       0
47821       0
47822       0
47823       0
47824       0
47825       0
47826       0
47827       0
47828       0
47829       0
47830       0
47831       0
47832       0
47833       0
47834       0
47835       0
47836       0
Name: open_int, Length: 207, dtype: object

In [47]:
# drop zero
nifty = bhav_copy[(bhav_copy['key']=='ACC_FUTSTK_0.0_XX_1') & (~(bhav_copy['open_int']==0))]

In [52]:
co,mi,ma,avg,per95,per97,per5,per3 = nifty['open_int'].astype(int).describe(
            percentiles=[.95,.97,.05,.03]).loc[['count','min','max','mean','95%','97%','5%','3%']].values  

227

In [69]:
bhav_copy[(bhav_copy['key']=='ACC_FUTSTK_0.0_XX_1') & (bhav_copy['price_date']==cassandra.util.Date(datetime.datetime.now()) ) ]

Empty DataFrame
Columns: [symbol, instrument, open_int, key, price_date, expiry_dt]
Index: []

In [124]:
if len(bhav_copy[(bhav_copy['key']=='ACC_FUTSTK_0.0_XX_1') &
                 (bhav_copy['price_date']==sorted(bhav_copy['price_date'], reverse = True)[0]) ]) == 0:    
    
    logging.info('{0} is newly traded/removed from FNO...'.format(key.split("_")[0]))
    current_OI = 0
    
else:
    current_OI = int(bhav_copy[(bhav_copy['key']=='ACC_FUTSTK_0.0_XX_1') &
                               (bhav_copy['price_date']==sorted(bhav_copy['price_date'], reverse = True)[0])]['open_int'].values[0])
    

    
    

In [123]:
current_OI

1225200

In [86]:
bhav_copy[(bhav_copy['key']==key) & (bhav_copy['price_date']==cassandra.util.Date(datetime.datetime.now()-datetime.timedelta(days=1)) ) ]

Empty DataFrame
Columns: [symbol, instrument, open_int, key, price_date, expiry_dt]
Index: []

In [106]:
bhav_copy[(bhav_copy['key']=='ACC_FUTSTK_0.0_XX_1') &
          (bhav_copy['price_date']==sorted(bhav_copy['price_date'], reverse = True)[0]) ]


symbol instrument open_int                  key  price_date   expiry_dt
226    ACC     FUTSTK  1225200  ACC_FUTSTK_0.0_XX_1  2018-11-26  2018-11-29

In [112]:
nifty = bhav_copy[bhav_copy['key']=='NIFTYCPSE_FUTIDX_0.0_XX_1']


Date(17829)

In [115]:
nifty[nifty['price_date']==sorted(nifty['price_date'], reverse = True)[0] ]

symbol instrument open_int                        key  price_date  \
47836  NIFTYCPSE     FUTIDX        0  NIFTYCPSE_FUTIDX_0.0_XX_1  2018-10-25   

        expiry_dt  
47836  2018-10-25

In [118]:
bhav_copy[bhav_copy['price_date']==sorted(bhav_copy['price_date'], reverse = True)[0]]

symbol instrument  open_int                         key  \
226           ACC     FUTSTK   1225200         ACC_FUTSTK_0.0_XX_1   
453      ADANIENT     FUTSTK  15040000    ADANIENT_FUTSTK_0.0_XX_1   
680    ADANIPORTS     FUTSTK  14870000  ADANIPORTS_FUTSTK_0.0_XX_1   
907    ADANIPOWER     FUTSTK  66800000  ADANIPOWER_FUTSTK_0.0_XX_1   
1134   AJANTPHARM     FUTSTK    612500  AJANTPHARM_FUTSTK_0.0_XX_1   
1361         ALBK     FUTSTK  12848000        ALBK_FUTSTK_0.0_XX_1   
1588   AMARAJABAT     FUTSTK   1194900  AMARAJABAT_FUTSTK_0.0_XX_1   
1815    AMBUJACEM     FUTSTK  13837500   AMBUJACEM_FUTSTK_0.0_XX_1   
2042   APOLLOHOSP     FUTSTK   1334000  APOLLOHOSP_FUTSTK_0.0_XX_1   
2269   APOLLOTYRE     FUTSTK  10011000  APOLLOTYRE_FUTSTK_0.0_XX_1   
2496       ARVIND     FUTSTK   6724000      ARVIND_FUTSTK_0.0_XX_1   
2723     ASHOKLEY     FUTSTK  49828000    ASHOKLEY_FUTSTK_0.0_XX_1   
2950   ASIANPAINT     FUTSTK   5123400  ASIANPAINT_FUTSTK_0.0_XX_1   
3177   AUROPHARMA     FUTSTK  14379000  AUROPHARMA_FUTSTK_0.0_XX_1   
3404     AXISBANK     FUTSTK  30603600    AXISBANK_FUTSTK_0.0_XX_1   
3631   BAJAJ-AUTO     FUTSTK   1703000  BAJAJ-AUTO_FUTSTK_0.0_XX_1   
3858   BAJFINANCE     FUTSTK   5395750  BAJFINANCE_FUTSTK_0.0_XX_1   
4085   BANKBARODA     FUTSTK  56696000  BANKBARODA_FUTSTK_0.0_XX_1   
4312    BANKINDIA     FUTSTK  17664000   BANKINDIA_FUTSTK_0.0_XX_1   
4539    BATAINDIA     FUTSTK   1683000   BATAINDIA_FUTSTK_0.0_XX_1   
4766          BEL     FUTSTK  17226000         BEL_FUTSTK_0.0_XX_1   
4993         BEML     FUTSTK    994000        BEML_FUTSTK_0.0_XX_1   
5220    BHARATFIN     FUTSTK   6339500   BHARATFIN_FUTSTK_0.0_XX_1   
5447   BHARATFORG     FUTSTK   5887200  BHARATFORG_FUTSTK_0.0_XX_1   
5674   BHARTIARTL     FUTSTK  34413100  BHARTIARTL_FUTSTK_0.0_XX_1   
5901         BHEL     FUTSTK  42427500        BHEL_FUTSTK_0.0_XX_1   
6128       BIOCON     FUTSTK   4543200      BIOCON_FUTSTK_0.0_XX_1   
6355     BOSCHLTD     FUTSTK    138780    BOSCHLTD_FUTSTK_0.0_XX_1   
6582         BPCL     FUTSTK  10425600        BPCL_FUTSTK_0.0_XX_1   
6809    BRITANNIA     FUTSTK   1316000   BRITANNIA_FUTSTK_0.0_XX_1   
...           ...        ...       ...                         ...   
40632        MFSL     FUTSTK   1845600        MFSL_FUTSTK_0.0_XX_1   
40859     INDIANB     FUTSTK   4546000     INDIANB_FUTSTK_0.0_XX_1   
41086  BALKRISIND     FUTSTK   1940000  BALKRISIND_FUTSTK_0.0_XX_1   
41313        NBCC     FUTSTK  20052000        NBCC_FUTSTK_0.0_XX_1   
41540         MGL     FUTSTK   1081200         MGL_FUTSTK_0.0_XX_1   
41767  BERGEPAINT     FUTSTK   2046000  BERGEPAINT_FUTSTK_0.0_XX_1   
41994  CANFINHOME     FUTSTK   2937500  CANFINHOME_FUTSTK_0.0_XX_1   
42221    CHOLAFIN     FUTSTK    516000    CHOLAFIN_FUTSTK_0.0_XX_1   
42448        GSFC     FUTSTK  11313000        GSFC_FUTSTK_0.0_XX_1   
42675         MCX     FUTSTK   2701300         MCX_FUTSTK_0.0_XX_1   
42902   NESTLEIND     FUTSTK    271650   NESTLEIND_FUTSTK_0.0_XX_1   
43129      VGUARD     FUTSTK   2511000      VGUARD_FUTSTK_0.0_XX_1   
43356  GODFRYPHLP     FUTSTK    417200  GODFRYPHLP_FUTSTK_0.0_XX_1   
43583  BAJAJFINSV     FUTSTK    561375  BAJAJFINSV_FUTSTK_0.0_XX_1   
43810        MRPL     FUTSTK   2632500        MRPL_FUTSTK_0.0_XX_1   
44037     RAYMOND     FUTSTK   3052800     RAYMOND_FUTSTK_0.0_XX_1   
44264         HCC     FUTSTK  13903659         HCC_FUTSTK_0.0_XX_1   
44491    RAMCOCEM     FUTSTK    540000    RAMCOCEM_FUTSTK_0.0_XX_1   
44718  NATIONALUM     FUTSTK  22096000  NATIONALUM_FUTSTK_0.0_XX_1   
44945     RBLBANK     FUTSTK   5112000     RBLBANK_FUTSTK_0.0_XX_1   
45172  CHENNPETRO     FUTSTK    843000  CHENNPETRO_FUTSTK_0.0_XX_1   
45399    SREINFRA     FUTSTK   9310000    SREINFRA_FUTSTK_0.0_XX_1   
45626  MANAPPURAM     FUTSTK  10968000  MANAPPURAM_FUTSTK_0.0_XX_1   
45853  ICICIPRULI     FUTSTK   3188900  ICICIPRULI_FUTSTK_0.0_XX_1   
46080   REPCOHOME     FUTSTK    502200   REPCOHOME_FUTSTK_0.0_XX_1   
46307  KAJARIACER    

In [137]:
colour_coding = pd.read_excel(output_dir+'Descriptive_report.xlsx')

In [139]:
from xlwt import Workbook
import xlwt
book = Workbook()
sheet1 = book.add_sheet('Sheet 1')
book.add_sheet('Sheet 2')
for i in range(0, 100):
    st = xlwt.easyxf('pattern: pattern solid;')
    st.pattern.pattern_fore_colour = i
    sheet1.write(i % 24, i / 24, 'Test text',st)
book.save('simple.xls')

In [142]:
import xlsxwriter
workbook = xlsxwriter.Workbook(output_dir+'Descriptive_report.xlsx')

In [144]:
# Add a format. Light red fill with dark red text.
format1 = workbook.add_format({'bg_color': '#FFC7CE'})

# Add a format. Green fill with dark green text.
format2 = workbook.add_format({'bg_color': '#C6EFCE'})


In [145]:
for row,rowdata in enumerate(workbook):
    print rowdata

TypeError: 'Workbook' object is not iterable

In [251]:
import xlsxwriter

#colour_coding = load_workbook(output_dir+'Descriptive_report.xlsx')


# Add a format. Light red fill with dark red text.
format1 = workbook.add_format({'bg_color': '#FFC7CE'})

# Add a format. Green fill with dark green text.
format2 = workbook.add_format({'bg_color': '#C6EFCE'})

writer = pd.ExcelWriter('fancy.xlsx', engine='xlsxwriter')
result.to_excel(writer, index=False, sheet_name='report')

workbook  = writer.book
worksheet = writer.sheets['report']

In [252]:
worksheet.conditional_format('A2:K300',
                             {'type':     'cell',
                                        'criteria': '>=',
                                        'value':    50,
                                        'format':   format1})

writer.save()



In [259]:
import pandas as pd
import numpy as np

np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[0, 2] = np.nan

def highlight_max(s):
    '''
    highlight the maximum in a S
    '''
    is_max = s == s.max()
    return ['background-color: light-blue' if v else '' for v in is_max]

df = df.style.apply(highlight_max)

In [260]:
df.to_excel('tessss.xlsx')

X:\Anaconda\lib\site-packages\pandas\io\formats\excel.py:312: CSSWarning: Unhandled color format: 'light-blue'
  CSSWarning)


In [436]:
def highlight_greater(row):
    color = 'white'
    if row['Current_OI'] > row['95%']:
        color = '#2E86C1'
    elif row['Current_OI'] > row['Avg']:
        color = '#D6EAF8'

    background = ['background-color: {}'.format(color) for i in row]
    
    return background

final = result.style.apply(highlight_greater, axis=1)

In [293]:
final.to_excel('tess.xlsx')

In [437]:
final

In [447]:
import matplotlib.pyplot as plt
from matplotlib import colors

def background_gradient(s, m, M, cmap='RdYlGn', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

final = result.style.applymap(background_gradient,subset=pd.IndexSlice[:, ['Min']],
               
               m=result['Min'].min(),
               M=result['Min'].max())

In [448]:
final

ValueError: (u'Invalid RGBA argument: 0.6624375240292195', u'occurred at index Min')

In [450]:
final = result['Min']

In [457]:
result.loc[:,3].style.background_gradient(cmap='viridis', axis=1)

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [3] of <type 'int'>

In [493]:
result1 = pd.read_excel('X:/Data_Analytics/FNO/Output/Descriptive_report.xlsx')


In [494]:
result1 = result1.sort_values(ascending=False, by=['Range_percentile_%'])


In [497]:
result1.style.background_gradient(cmap='RdBu', subset = pd.IndexSlice[:, ['Range_percentile_%']])

In [111]:
import pandas as pd
#from dateutil import parser
from cassandra.cluster import Cluster
#import cassandra
import logging 
from time import time
import datetime
import Cassandra_dumper
#import numpy as np 

download_dir = "X:\\Data_Analytics\\FNO\\Download\\"
master_dir = "X:\\Data_Analytics\\FNO\\Master\\"
log_path = "X:\\Data_Analytics\\FNO\\"
output_dir = "X:\Data_Analytics\FNO\Output\\"

# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")


# Define a pandas factory to read the cassandra data into pandas dataframe:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)


def instu(row):
    row = row.split('_')
    return row[1]

def sym(row):
    row = row.split('_')
    return row[0]

def descriptive_func(bhav_copy,keys):
    '''Function to get Min Max Avg 95% 97% 5% 3% for individual stocks'''      
 
    logging.info('Generating report for OPEN_INT...Min/Max/Avg/95%/97%/5%/3%')
    result = []
    # current date
    current_date = sorted(bhav_copy['price_date'], reverse = True)[0]
    logging.info('Processing for current date: {0}'.format(current_date))
    
    for key in keys:
        current_OI = 0
        
        # drop rows with zero OI
        stock = bhav_copy[(bhav_copy['key']==key) & (~(bhav_copy['open_int']==0))]
        #current OI
        if len(stock[stock['price_date']==current_date ]) == 0:  
            logging.info('{0} is newly traded/removed from FNO...'.format(key.split("_")[0]))
            current_OI = 0
        else:
            current_OI = int(stock[stock['price_date']==current_date]['open_int'].values[0])
        
        Range_percentile = 0       
        try:
            # convert to int and get the percentiles with describe method
            count,mi,ma,avg,per95,per97,per5,per3 = stock['open_int'].astype(int).describe(
                percentiles=[.95,.97,.05,.03]).loc[['count','min','max','mean','95%','97%','5%','3%']].values 
            
            if ma-mi == 0:
                Range_percentile = 0
            else:
                #Range_percentile = "{0:.0%}".format( (current_OI - mi) / (ma-mi) )
                Range_percentile = round((current_OI - mi)*100 / (ma-mi),2)
                
            
            
        except Exception as e:
            logging.info('{0}'.format(e))
            logging.info('{0} Traded newly with OI zero...'.format(key.split("_")[0]))
        
        
        result.append([key,count,mi,ma,avg,per95,per97,per5,per3,current_OI,Range_percentile])                         
                          
                      
    result = pd.DataFrame(result, columns=['key','Days_traded','Min','Max','Avg','95%','97%','5%','3%','Current_OI','Range_percentile_%']) 
    # get symbol from key
    result['SYMBOL'] = result.apply(lambda row: sym(row['key']), axis=1)
    # get instruments
    result['INSTRUMENT'] = result.apply(lambda row: instu(row['key']), axis=1)
    # drop key
    result.drop('key',inplace=True, axis=1)
    # write the report to excel 
    result = result[['SYMBOL','INSTRUMENT','Days_traded','Current_OI','Range_percentile_%','Min','Max','Avg','95%','97%','5%','3%']]
    
    #result = result.style.apply(highlight_greater, axis=1)
    #result = result.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['Range_percentile']])
    #result = result.costyle.apply(background_gradient, axis = 1, columns=['Range_percentile'])
    result = result.sort_values(ascending=False, by=['Range_percentile_%'])
    result = result.style.background_gradient(cmap='RdBu', subset = pd.IndexSlice[:, ['Range_percentile_%']])

    #result.to_excel(output_dir+'Descriptive_report.xlsx', index=False)
    return result
    logging.info('Descriptive report generated successfully in excel...')
    
    
    
    
    
    
def cassandra_get_df(keys):
    '''Function to fetch values from cassandra db'''
    logging.info('Fetching values from cassandra')
    cluster = Cluster(['127.0.0.1'])
    logging.info('Cassandra Cluster connected...')
    # connect to your keyspace and create a session using which u can execute cql commands 
    session = cluster.connect('test_df')
    logging.info('Using test_df keyspace')
    #define rows fetched from cassandra to be a pandas dataframe
    session.row_factory = pandas_factory
    session.default_fetch_size = None

    # Process for date range 
    #from_date = parser.parse(raw_input('Enter from price date :'))
    #to_date = parser.parse(raw_input('Enter to price date :'))
    
    # create empty dataframe 
    bhav_copy = pd.DataFrame()

    for key in keys:

        query = 'SELECT symbol,instrument,open_int,key,price_date,expiry_dt FROM test_bhavcopy WHERE price_date >= \'{0}\' AND price_date <= \'{1}\' AND key = \'{2}\'ALLOW FILTERING;'.format(
                    datetime.datetime.now().date() - datetime.timedelta(days=365),datetime.datetime.now().date(),key)
        rows = session.execute(query, timeout = None)
        rows = rows._current_rows

        if len(rows)==0:
            pass
        else :
            bhav_copy = bhav_copy.append(rows,ignore_index=True)
    
    return bhav_copy

        
def dataframe_merger(current_month,near_month):
    '''Function to merge current_OI and near month_OI dataframes'''
    
    # process current_oi df to perform merge on
    current_month['index_key'] = current_month.apply(lambda row: str(row['price_date']), axis=1)
    current_month['merge'] = current_month['key']+'_'+current_month['index_key']
    current_month.drop(columns=['index_key'], inplace=True)

    # process nea_month_oi to perfrom merge on
    near_month['index_key'] = near_month.apply(lambda row: str(row['price_date']), axis=1)
    near_month['merge'] = near_month['key']+'_'+near_month['index_key']
    near_month.drop(columns=['price_date','index_key','key'], inplace=True)

    # perform merge
    df = current_month.merge(near_month, on='merge', how='left')
    # replace NaN with zeros for symbols that werent traded on near_month_OI
    df.fillna(0, inplace=True)
    # total_OI
    df['Total_OI'] = df['open_int']+df['near_month_OI']
    # drop merger key
    df.drop(columns=['merge'], inplace=True)
    return df 
    

In [112]:
logging.info('Read master file to fetch Active FNO symbols and generate keys...')
keys = pd.read_excel(master_dir+'MasterData.xlsx')
keys = keys[keys['IsActiveFNO']==True]
keys.loc[keys['Type']=='SSF','instrument'] = 'FUTSTK'
keys.loc[keys['Type']=='Index','instrument'] = 'FUTIDX'
keys = keys['SYMBOL']+"_"+keys['instrument']+"_0.0_XX_1"

# Fetch values from cassandra for Current_OI
current_OI_df = cassandra_get_df(keys)

# get near month OI from cassandra
keys_near_month = keys.str.replace('_1','_2')
near_month_df = cassandra_get_df(keys_near_month)
near_month_df = near_month_df[['key','open_int','price_date']]          
        
#replace back to og key to perfrom merge
near_month_df['key'] = near_month_df['key'].str.replace('_2','_1')
#rename open_int into near_month_oi
near_month_df.rename(columns = {'open_int':'near_month_OI'},inplace=True)

bhav_copy = dataframe_merger(current_OI_df, near_month_df)





logging.info('Calling descriptive func to process descriptive stats') 
descriptive_func(bhav_copy,keys)

In [113]:

    
result = []
# current date
current_date = sorted(bhav_copy['price_date'], reverse = True)[0]
logging.info('Processing for current date: {0}'.format(current_date.date()))
    
for key in keys:
    current_OI = 0
    near_month_OI = 0
        
    # drop rows with zero current OI
    stock = bhav_copy[(bhav_copy['key']==key) & (~(bhav_copy['open_int']==0))]
    #current OI
    if len(stock[stock['price_date']==current_date ]) == 0:  
        logging.info('{0} is newly traded/removed from FNO...'.format(key.split("_")[0]))
        current_OI = 0
        near_month_OI = 0
        total_OI = 0
    else:
        current_OI = int(stock[stock['price_date']==current_date]['open_int'].values[0])
        near_month_OI = int(stock[stock['price_date']==current_date]['near_month_OI'].values[0])
        total_OI = current_OI + near_month_OI
        
    Range_percentile = 0       
    try:
        # convert to int and get the percentiles with describe method
        count,mi,ma,avg,per95,per97,per5,per3 = stock['Total_OI'].astype(int).describe(
            percentiles=[.95,.97,.05,.03]).loc[['count','min','max','mean','95%','97%','5%','3%']].values 
            
        if ma-mi == 0:
            Range_percentile = 0
        else:
            #Range_percentile = "{0:.0%}".format( (current_OI - mi) / (ma-mi) )
            Range_percentile = round((total_OI - mi)*100 / (ma-mi),2)
                
            
            
    except Exception as e:
        logging.info('{0}'.format(e))
        logging.info('{0} Traded newly with OI zero...'.format(key.split("_")[0]))
        
        
    result.append([key,count,mi,ma,avg,per95,per97,per5,per3,current_OI,Range_percentile,near_month_OI, total_OI])                         
                          
                      
result = pd.DataFrame(result, columns=['key','Days_traded','Min','Max','Avg','95%','97%','5%','3%','Current_OI','Range_percentile_%','Near_month_OI','Total_OI']) 
# get symbol from key
result['SYMBOL'] = result.apply(lambda row: sym(row['key']), axis=1)
# get instruments
result['INSTRUMENT'] = result.apply(lambda row: instu(row['key']), axis=1)
# drop key
result.drop('key',inplace=True, axis=1)
# write the report to excel 
result = result[['SYMBOL','INSTRUMENT','Days_traded','Total_OI','Range_percentile_%','Min','Max','Avg','95%','97%','5%','3%','Current_OI','Near_month_OI']]
    
#result = result.style.apply(highlight_greater, axis=1)
#result = result.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['Range_percentile']])
#result = result.costyle.apply(background_gradient, axis = 1, columns=['Range_percentile'])
result = result.sort_values(ascending=False, by=['Range_percentile_%'])
result = result.style.background_gradient(cmap='RdBu', subset = pd.IndexSlice[:, ['Range_percentile_%']])

#result.to_excel(output_dir+'Descriptive_report.xlsx', index=False)
#return result
logging.info('Descriptive report generated successfully in excel...')

In [114]:
result

In [4]:
import dateutil.parser as dparser

In [9]:
parser.parse(filename[2:4]+' '+filename[4:7]+' '+filename[7:10]).date()

datetime.date(201, 12, 17)

In [69]:
import pandas as pd
#from dateutil import parser
from cassandra.cluster import Cluster
#import cassandra
import logging 
from time import time
import datetime
import Cassandra_dumper
#import numpy as np 
import shutil
import plotly.offline as py
import plotly.graph_objs as go


download_dir = "X:\\Data_Analytics\\FNO\\Download\\"
master_dir = "X:\\Data_Analytics\\FNO\\Master\\"
log_path = "X:\\Data_Analytics\\FNO\\"
output_dir = "X:\Data_Analytics\FNO\Output\\"
report_dir = "X:\\Data_Analytics\\Emails\\Output\\Daily_Descriptive_reports\\"
# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")


# Define a pandas factory to read the cassandra data into pandas dataframe:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)


def instu(row):
    row = row.split('_')
    return row[1]

def sym(row):
    row = row.split('_')
    return row[0]

def descriptive_func(bhav_copy,keys):
    '''Function to get Min Max Avg 95% 97% 5% 3% for individual stocks'''      
 
    logging.info('Generating report for OPEN_INT...Min/Max/Avg/95%/97%/5%/3%')
    result = []
    # current date
    current_date = sorted(bhav_copy['price_date'], reverse = True)[0]
    logging.info('Processing for current date: {0}'.format(current_date.date()))
        
    for key in keys:
        current_OI = 0
        near_month_OI = 0
            
        # drop rows with zero current OI
        stock = bhav_copy[(bhav_copy['key']==key) & (~(bhav_copy['open_int']==0))]
        #current OI
        if len(stock[stock['price_date']==current_date ]) == 0:  
            logging.info('{0} is newly traded/removed from FNO...'.format(key.split("_")[0]))
            current_OI = 0
            near_month_OI = 0
            total_OI = 0
        else:
            current_OI = int(stock[stock['price_date']==current_date]['open_int'].values[0])
            near_month_OI = int(stock[stock['price_date']==current_date]['near_month_OI'].values[0])
            total_OI = current_OI + near_month_OI
            
        Range_percentile = 0       
        try:
            # convert to int and get the percentiles with describe method
            count,mi,ma,avg,per95,per97,per5,per3 = stock['Total_OI'].astype(int).describe(
                percentiles=[.95,.97,.05,.03]).loc[['count','min','max','mean','95%','97%','5%','3%']].values 
                
            if ma-mi == 0:
                Range_percentile = 0
            else:
                #Range_percentile = "{0:.0%}".format( (current_OI - mi) / (ma-mi) )
                Range_percentile = round((total_OI - mi)*100 / (ma-mi),2)
                    
                
                
        except Exception as e:
            logging.info('{0}'.format(e))
            logging.info('{0} Traded newly with OI zero...'.format(key.split("_")[0]))
            
            
        result.append([key,count,mi,ma,avg,per95,per97,per5,per3,current_OI,Range_percentile,near_month_OI, total_OI])                         
                              
                          
    result = pd.DataFrame(result, columns=['key','Days_traded','Min','Max','Avg','95%','97%','5%','3%','Current_OI','Range_percentile_%','Near_month_OI','Total_OI']) 
    # get symbol from key
    result['SYMBOL'] = result.apply(lambda row: sym(row['key']), axis=1)
    # get instruments
    result['INSTRUMENT'] = result.apply(lambda row: instu(row['key']), axis=1)
    # drop key
    result.drop('key',inplace=True, axis=1)
    # write the report to excel 
    result = result[['SYMBOL','INSTRUMENT','Days_traded','Total_OI','Range_percentile_%','Min','Max','Avg','95%','97%','5%','3%','Current_OI','Near_month_OI']]
        
    #result = result.style.apply(highlight_greater, axis=1)
    #result = result.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['Range_percentile']])
    #result = result.costyle.apply(background_gradient, axis = 1, columns=['Range_percentile'])
    result = result.sort_values(ascending=False, by=['Range_percentile_%'])
    result = result.style.background_gradient(cmap='RdBu', subset = pd.IndexSlice[:, ['Range_percentile_%']])
    
    result.to_excel(output_dir+'Descriptive_report.xlsx', index=False)
    #return result
    logging.info('Descriptive report generated successfully in excel...')
    
    
    
    
    
def cassandra_get_df(keys):
    '''Function to fetch values from cassandra db'''
    logging.info('Fetching values from cassandra')
    cluster = Cluster(['127.0.0.1'])
    logging.info('Cassandra Cluster connected...')
    # connect to your keyspace and create a session using which u can execute cql commands 
    session = cluster.connect('test_df')
    logging.info('Using test_df keyspace')
    #define rows fetched from cassandra to be a pandas dataframe
    session.row_factory = pandas_factory
    session.default_fetch_size = None    
    
    # create empty dataframe 
    bhav_copy = pd.DataFrame()

    for key in keys:

        query = 'SELECT symbol,instrument,open_int,key,price_date,expiry_dt FROM test_bhavcopy WHERE price_date >= \'{0}\' AND price_date <= \'{1}\' AND key = \'{2}\'ALLOW FILTERING;'.format(
                    datetime.datetime.now().date() - datetime.timedelta(days=365),datetime.datetime.now().date(),key)
        rows = session.execute(query, timeout = None)
        rows = rows._current_rows

        if len(rows)==0:
            pass
        else :
            bhav_copy = bhav_copy.append(rows,ignore_index=True)
    bhav_copy.to_excel('test.xlsx')
    return bhav_copy

        
def dataframe_merger(current_month,near_month):
    '''Function to merge current_OI and near month_OI dataframes'''
    
    # process current_oi df to perform merge on
    # key is key_price date
    current_month['index_key'] = current_month.apply(lambda row: str(row['price_date']), axis=1)
    current_month['merge'] = current_month['key']+'_'+current_month['index_key']
    current_month.drop(columns=['index_key'], inplace=True)

    # process nea_month_oi to perfrom merge on
    near_month['index_key'] = near_month.apply(lambda row: str(row['price_date']), axis=1)
    near_month['merge'] = near_month['key']+'_'+near_month['index_key']
    near_month.drop(columns=['price_date','index_key','key'], inplace=True)

    # perform merge
    df = current_month.merge(near_month, on='merge', how='left')
    # replace NaN with zeros for symbols that werent traded on near_month_OI
    df.fillna(0, inplace=True)
    # total_OI
    df['Total_OI'] = df['open_int']+df['near_month_OI']
    # drop merger key
    df.drop(columns=['merge'], inplace=True)
    
    
    return df 
    



def total_OI_and_nifty_graph(bhav_copy):
    '''Func to calculate total oi for each day, and plot the graph with nifty'''
    # get the df with data of last one year for current and near month expiry and calc total oi for individual price date
    price_dates = list(set(bhav_copy['price_date']))
    result = []
    # calc total OI and fetch nifty 
    
    for price_date in price_dates:
        # filter on price date 
        df = bhav_copy[bhav_copy['price_date']==price_date.date()]
        # total OI
        total_oi = df['Total_OI'].sum()
        # nifty total oi
        try:
            nifty_val = df[df['symbol']=='NIFTY']['Total_OI'].values[0]
        except Exception as e:
            print 'Nifty exception:  {0}'.format(e)
            nifty_val = 0
            
        result.append([price_date.date(), total_oi, nifty_val])
        
    # convert list to dataframe 
    result = pd.DataFrame(data=result, columns=['Price_date', 'Total_OI', 'NIFTY'])
    result.set_index('Price_date', inplace= True)
    return result
        
    
    


def main():
    '''Main function'''
    
    Cassandra_dumper.cassandra_dumper()
    
    logging.info('Read master file to fetch Active FNO symbols and generate keys...')
    keys = pd.read_excel(master_dir+'MasterData.xlsx')
    keys = keys[keys['IsActiveFNO']==True]
    keys.loc[keys['Type']=='SSF','instrument'] = 'FUTSTK'
    keys.loc[keys['Type']=='Index','instrument'] = 'FUTIDX'
    keys = keys['SYMBOL']+"_"+keys['instrument']+"_0.0_XX_1"
    
    # Fetch values from cassandra for Current_OI
    current_OI_df = cassandra_get_df(keys)
    
    # get near month OI from cassandra
    keys_near_month = keys.str.replace('_1','_2')
    near_month_df = cassandra_get_df(keys_near_month)
    near_month_df = near_month_df[['key','open_int','price_date']]          
            
    #replace back to og key to perfrom merge
    near_month_df['key'] = near_month_df['key'].str.replace('_2','_1')
    #rename open_int into near_month_oi
    near_month_df.rename(columns = {'open_int':'near_month_OI'},inplace=True)
    
    bhav_copy = dataframe_merger(current_OI_df, near_month_df)
    return bhav_copy
    # cal func to get total oi and nifty graph 
    total_oi_and_nifty_df = total_OI_and_nifty_graph(bhav_copy)
    return total_oi_and_nifty_df
    
    logging.info('Calling descriptive func to process descriptive stats') 
    descriptive_func(bhav_copy,keys)
    shutil.copyfile(output_dir+'Descriptive_report.xlsx', report_dir+'Descriptive_report.xlsx')
    
    
bhav_copy  = pd.DataFrame()    
start_time = time()

if __name__ == '__main__':
    bhav_copy = main()

end_time = time()

logging.info('Time taken to process :'.format(end_time - start_time))
print "Execution time: {0} Seconds.... ".format(end_time - start_time)


          


Execution time: 107.406999826 Seconds.... 


In [105]:
price_dates = list(set(bhav_copy['price_date']))
result = []
# calc total OI and fetch nifty 
    
for price_date in price_dates:
    df = bhav_copy[bhav_copy['price_date']==price_date.date()]
    # total OI
    total_oi = df['Total_OI'].sum()
    # nifty total oi
    try:
        nifty_val = df[df['symbol']=='NIFTY']['Total_OI'].values[0]
        
    except Exception as e:
        print 'Nifty exception:  {0}'.format(e)
        nifty_val = 0
            
    result.append([price_date.date(), total_oi, nifty_val])
        
# convert list to dataframe 
result = pd.DataFrame(data=result, columns=['Price_date', 'Total_OI', 'NIFTY'])

# create a trace for x and y 
trace1 = go.Scatter(x=result['Price_date'], y=result['Total_OI'], mode='lines+markers', name='SUM_(Total_OI) ') 
trac2 = go.Scatter(x=result['Price_date'], y=result['NIFTY'], mode = 'lines+markers', name = 'NIFTY (Current_OI + Nearmonth_OI)')

data = [trace1, trac2]

py.plot(data,image='jpeg', image_filename='total_OI_nifty_graph')




In [114]:
result['diff'] = result['Total_OI'] - result['NIFTY']



In [115]:
result.head()

Price_date    Total_OI     NIFTY        diff
0  2017-12-19  5168239444  21923625  5146315819
1  2017-12-20  5217354151  23113875  5194240276
2  2017-12-21  5367131472  23805450  5343326022
3  2017-12-22  5375088563  25339650  5349748913
4  2017-12-26  5447624469  27320250  5420304219